# Basic EDA

- competition 코드 공유에 공개되어있는 EDA를 참고하여 확인한 내용입니다.
- [주차수요 예측 EDA & Catboost Baseline(LB: 118.6101)](https://dacon.io/competitions/official/235745/codeshare/2851?page=1&dtype=recent)


## Import module

In [14]:
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm

import matplotlib
from matplotlib import font_manager, rc
from matplotlib import pyplot as plt
import platform

if platform.system() == 'Window':
    font_name = font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
    rc('font', family=font_name)
else:
    rc('font', family='AppleChthic')

matplotlib.rcParams['axes.unicode_minus'] = False

## Data Search

### Data Load & Basic Search

In [6]:
DATA_ROOT = ''
DATA_ROOT = os.path.join(DATA_ROOT, "../../../competition_data/parking_data/")

TRAIN_ROOT = os.path.join(DATA_ROOT, "train.csv")
TEST_ROOT = os.path.join(DATA_ROOT, "test.csv")
AGE_GENDER_INFO_ROOT = os.path.join(DATA_ROOT, "age_gender_info.csv")

print(f"DATA_ROOT : {DATA_ROOT}")
print(f"TRAIN_ROOT : {TRAIN_ROOT}")
print(f"TEST_ROOT : {TEST_ROOT}")
print(F"SUBMISSION_ROOT : {AGE_GENDER_INFO_ROOT}")

DATA_ROOT : ../../../competition_data/parking_data/
TRAIN_ROOT : ../../../competition_data/parking_data/train.csv
TEST_ROOT : ../../../competition_data/parking_data/test.csv
SUBMISSION_ROOT : ../../../competition_data/parking_data/age_gender_info.csv


In [7]:
train = pd.read_csv(TRAIN_ROOT)
test = pd.read_csv(TEST_ROOT)
age_gender_info = pd.read_csv(AGE_GENDER_INFO_ROOT)

print("Data Loaded!")

Data Loaded!


- train에서 Null을 포함하는 features
    - 임대보증금
    - 임대료
    - 도보 10분거리 내 지하철역 수(환ㅅ으노선 수 반영)
    - 도보 10분거리 내 버스정류장 수

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2952 entries, 0 to 2951
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          2952 non-null   object 
 1   총세대수                          2952 non-null   int64  
 2   임대건물구분                        2952 non-null   object 
 3   지역                            2952 non-null   object 
 4   공급유형                          2952 non-null   object 
 5   전용면적                          2952 non-null   float64
 6   전용면적별세대수                      2952 non-null   int64  
 7   공가수                           2952 non-null   float64
 8   자격유형                          2952 non-null   object 
 9   임대보증금                         2383 non-null   object 
 10  임대료                           2383 non-null   object 
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  2741 non-null   float64
 12  도보 10분거리 내 버스정류장 수            2948 non-null   float64
 13  단지내

- test에서 Null을 포함하는 features
    - 자격유형
    - 임대보증금
    - 임대료
    - 도보 10분거리 내 지하철역 수(환승노선 수 반영)

In [9]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1022 entries, 0 to 1021
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          1022 non-null   object 
 1   총세대수                          1022 non-null   int64  
 2   임대건물구분                        1022 non-null   object 
 3   지역                            1022 non-null   object 
 4   공급유형                          1022 non-null   object 
 5   전용면적                          1022 non-null   float64
 6   전용면적별세대수                      1022 non-null   int64  
 7   공가수                           1022 non-null   float64
 8   자격유형                          1020 non-null   object 
 9   임대보증금                         842 non-null    object 
 10  임대료                           842 non-null    object 
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  980 non-null    float64
 12  도보 10분거리 내 버스정류장 수            1022 non-null   float64
 13  단지내

### 숫자이어야 하는 feature에서 숫자가 아닌 값 확인

- `임대보증금`, `임대료` 의 dtype이 object이다.

In [20]:
not_nums = []

for val in tqdm(train['임대보증금'].unique().tolist()):

    try :
        num = float(val)
    except :
        not_nums.append(val)

print(f"\nCharacter Values in 'train['임대보증금']' : \n{not_nums}\n")

not_nums = []

for val in tqdm(train['임대료'].unique().tolist()):

    try :
        num = float(val)
    except :
        not_nums.append(val)

print(f"\nCharacter Values in 'train['임대료']' : \n{not_nums}\n")

  0%|          | 0/958 [00:00<?, ?it/s]


Character Values in 'train['임대보증금']' : 
['-']



  0%|          | 0/996 [00:00<?, ?it/s]


Character Values in 'train['임대료']' : 
['-']



In [22]:
not_nums = []

for val in tqdm(test['임대보증금'].unique().tolist()):

    try :
        num = float(val)
    except :
        not_nums.append(val)

print(f"\nCharacter Values in 'test['임대보증금']' : \n{not_nums}\n")

not_nums = []

for val in tqdm(test['임대료'].unique().tolist()):

    try :
        num = float(val)
    except :
        not_nums.append(val)

print(f"\nCharacter Values in 'test['임대료']' : \n{not_nums}\n")

  0%|          | 0/405 [00:00<?, ?it/s]


Character Values in 'test['임대보증금']' : 
['-']



  0%|          | 0/405 [00:00<?, ?it/s]


Character Values in 'test['임대료']' : 
['-']



### '-'을 null로 바꾸고, dtype을 float로 변경

In [23]:
columns = ['임대보증금', '임대료']

for col in columns:
    train.loc[train[col] == '-', col] = np.nan
    test.loc[test[col] == '-', col] = np.nan

    train[col] = train[col].astype(float)
    test[col] = test[col].astype(float)

## NULL 값이 있는 변수 탐색

### `임대보증금`, `임대료`

- 임대보증금이 null이면 임대료도 null이다.
- '-' 인 경우도 모두 null로 만들어주었으므로,
    - null을 모두 0으로 만들자

In [38]:
columns = ['공급유형', '자격유형']
bases = ['임대보증금', '임대료']

for base in bases:
    print("========"*5)
    
    for col in columns:

        val_counts = train.loc[train[base].isnull(), col].value_counts()
        print(f"\n[Train '{col}' value counts based on '{base}'] : \n{val_counts}\n")
    
    print("========"*5)


[Train '공급유형' value counts based on '임대보증금'] : 
임대상가    562
국민임대      8
공공분양      7
행복주택      4
Name: 공급유형, dtype: int64


[Train '자격유형' value counts based on '임대보증금'] : 
D    569
H      8
K      4
Name: 자격유형, dtype: int64


[Train '공급유형' value counts based on '임대료'] : 
임대상가    562
장기전세      9
국민임대      8
공공분양      7
행복주택      4
Name: 공급유형, dtype: int64


[Train '자격유형' value counts based on '임대료'] : 
D    569
A      9
H      8
K      4
Name: 자격유형, dtype: int64



In [39]:
columns = ['공급유형', '자격유형']
bases = ['임대보증금', '임대료']

for base in bases:
    print("========"*5)
    
    for col in columns:

        val_counts = test.loc[test[base].isnull(), col].value_counts()
        print(f"\n[Test '{col}' value counts based on '{base}'] : \n{val_counts}\n")
    
    print("========"*5)


[Test '공급유형' value counts based on '임대보증금'] : 
임대상가    177
영구임대      5
행복주택      4
Name: 공급유형, dtype: int64


[Test '자격유형' value counts based on '임대보증금'] : 
D    180
L      4
C      2
Name: 자격유형, dtype: int64


[Test '공급유형' value counts based on '임대료'] : 
임대상가    177
영구임대      5
행복주택      4
Name: 공급유형, dtype: int64


[Test '자격유형' value counts based on '임대료'] : 
D    180
L      4
C      2
Name: 자격유형, dtype: int64



In [41]:
train[['임대보증금', '임대료']] = train[['임대보증금', '임대료']].fillna(0)
test[['임대보증금', '임대료']] = test[['임대보증금', '임대료']].fillna(0)

### `도보 10분거리 내 지하철역 수(환승노선 수 반영)`, `도보 10분거리 내 버스정류장 수`

- NULL 값을 0으로 대체

#### Train

In [46]:
subway_null_codes = train[train['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].isnull()].단지코드.unique()
print(f"Null이 포함된 example의 code 종류의 수 : {len(subway_null_codes)}")
print(f"해당 code인 example 수 (Null인 example 수) : {train.loc[train.단지코드.isin(subway_null_codes), '도보 10분거리 내 지하철역 수(환승노선 수 반영)'].value_counts(dropna=False).item()}")

Null이 포함된 example의 code 종류의 수 : 20
해당 code인 example 수 (Null인 example 수) : 211


In [48]:
bus_null_codes = train[train['도보 10분거리 내 버스정류장 수'].isnull()].단지코드.unique()
print(f"Null이 포함된 example의 code 종류의 수 : {len(bus_null_codes)}")
print(f"해당 code인 example 수 (Null인 example 수) : {train.loc[train.단지코드.isin(bus_null_codes), '도보 10분거리 내 버스정류장 수'].value_counts(dropna=False).item()}")

Null이 포함된 example의 code 종류의 수 : 1
해당 code인 example 수 (Null인 example 수) : 4


#### Test

In [47]:
subway_null_codes = test[test['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].isnull()].단지코드.unique()
print(f"Null이 포함된 example의 code 종류의 수 : {len(subway_null_codes)}")
print(f"해당 code인 example 수 (Null인 example 수) : {test.loc[test.단지코드.isin(subway_null_codes), '도보 10분거리 내 지하철역 수(환승노선 수 반영)'].value_counts(dropna=False).item()}")

Null이 포함된 example의 code 종류의 수 : 5
해당 code인 example 수 (Null인 example 수) : 42


#### fillna

In [49]:
cols = ['도보 10분거리 내 지하철역 수(환승노선 수 반영)', '도보 10분거리 내 버스정류장 수']
train[cols] = train[cols].fillna(0)
test[cols] = test[cols].fillna(0)

### `자격유형`

In [50]:
test[test.자격유형.isnull()]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
196,C2411,962,아파트,경상남도,국민임대,46.90,240,25.0,NaN,71950000.0,37470.0,0.0,2.0,840.0
258,C2253,1161,아파트,강원도,영구임대,26.37,745,0.0,NaN,2249000.0,44770.0,0.0,2.0,173.0


#### `C2411` 자격유형 양상

- A만 존재하므로 `A`로 채우면 될 듯 하다.

In [54]:
test.loc[test.단지코드 == 'C2411', '자격유형'].unique().tolist()

['A', nan]

In [58]:
test.loc[test.단지코드.isin(['C2411']) & test.자격유형.isnull(), '자격유형'] = 'A'

#### `C2253` 자격유형 양상

- 임대보증금, 임대료가 존재하면 자격유형이 C, 없으면 D로 설정되어있는 양상을 보인다.
- Nan이 있는 example은 존재하므로 `C`로 채우면 될 듯 하다.

In [55]:
test.loc[test.단지코드 == 'C2253', '자격유형'].unique().tolist()

[nan, 'C', 'D']

In [57]:
test[test.단지코드 == 'C2253']

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
258,C2253,1161,아파트,강원도,영구임대,26.37,745,0.0,NaN,2249000.0,44770.0,0.0,2.0,173.0
259,C2253,1161,아파트,강원도,영구임대,31.32,239,0.0,C,3731000.0,83020.0,0.0,2.0,173.0
260,C2253,1161,아파트,강원도,영구임대,31.32,149,0.0,C,3731000.0,83020.0,0.0,2.0,173.0
261,C2253,1161,상가,강원도,임대상가,13.77,1,0.0,D,0.0,0.0,0.0,2.0,173.0
262,C2253,1161,상가,강원도,임대상가,22.89,1,0.0,D,0.0,0.0,0.0,2.0,173.0
263,C2253,1161,상가,강원도,임대상가,22.91,1,0.0,D,0.0,0.0,0.0,2.0,173.0
264,C2253,1161,상가,강원도,임대상가,23.79,1,0.0,D,0.0,0.0,0.0,2.0,173.0
265,C2253,1161,상가,강원도,임대상가,23.79,1,0.0,D,0.0,0.0,0.0,2.0,173.0
266,C2253,1161,상가,강원도,임대상가,23.86,1,0.0,D,0.0,0.0,0.0,2.0,173.0
267,C2253,1161,상가,강원도,임대상가,23.86,1,0.0,D,0.0,0.0,0.0,2.0,173.0


In [59]:
test.loc[test.단지코드.isin(['C2253']) & test.자격유형.isnull(), '자격유형'] = 'C'

## 중복 확인

In [62]:
print(f"Train Shape : {train.shape}")
print(f"Drop Duplicates Train shape : {train.drop_duplicates().shape}")
print(f"Number of Duplicates in Train: {train.shape[0] - train.drop_duplicates().shape[0]}")

Train Shape : (2952, 15)
Drop Duplicates Train shape : (2632, 15)
Number of Duplicates in Train: 320


In [63]:
print(f"Test Shape : {test.shape}")
print(f"Drop Duplicates Test shape : {test.drop_duplicates().shape}")
print(f"Number of Duplicates in Test: {test.shape[0] - test.drop_duplicates().shape[0]}")

Test Shape : (1022, 14)
Drop Duplicates Test shape : (949, 14)
Number of Duplicates in Test: 73


### Drop Duplicates

In [64]:
train = train.drop_duplicates()
test = test.drop_duplicates()